In [1]:
import utils.connect_mods as conn
import utils.payload_construct_mods as paystruct

In [3]:
import yaml
import json

with open('device_config.yaml') as f:
    
    data = yaml.load(f, Loader=yaml.FullLoader)
    print(json.dumps(data, indent=4))

{
    "mqtt": {
        "oringpaas": {
            "client_id": "thing:syjmHv7z-",
            "host": "mqtt.paas.oringnet.cloud",
            "port": 1883,
            "username": "syjmHv7z-",
            "userpw": "ZCsC5XNCsk~mzrsy",
            "keepalive": 60,
            "topic": "$thing/syjmHv7z-/$data/",
            "publish_path": "sensorData"
        }
    }
}


In [3]:
a = conn.mqtt_connack(**data['mqtt']['oringpaas'])
a.connect()
a.payload_submit(json.dumps({"id":"voltage","value":'test'}))
a.client.disconnect()

connect to mqtt.paas.oringnet.cloud:1883
Connected with result code 0
userdata: None, mid: 1
is published: True


0

In [ ]:
import joblib
import time
import scipy.stats

a = conn.mqtt_connack(**data['mqtt']['oringpaas'])
a.connect()
a.client.loop_start()

while True:
    sim_data = {}
    cols = ['DO', 'BOD', 'SS', 'Nitrogen']
    for i in range(4):
        kde = joblib.load(f"./sim_kdes/kde{i}.pkl")
        _data = kde.resample(1).T[:,0]
        sim_data[cols[i]] = str(round(_data[0],2))
    
    a.payload_submit(json.dumps({"id":"fullDataPackt","value":str(sim_data)}))
    time.sleep(5.0)
# sim_data

In [4]:
a = conn.mqtt_connack(**data['mqtt']['oringpaas'])
dir(a)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'client',
 'client_id',
 'connect',
 'host',
 'keepalive',
 'on_connect',
 'on_message',
 'on_publish',
 'payload_submit',
 'port',
 'publish_path',
 'topic',
 'username',
 'userpw']

In [5]:
a.publish_path

{'dataBucket': 'sensorData',
 'dataShapes': ['voltage', 'cpuTemp', 'fullDataPackt']}

In [8]:
import os
os.environ

environ{'ELECTRON_RUN_AS_NODE': '1',
        'GJS_DEBUG_TOPICS': 'JS ERROR;JS LOG',
        'LC_TIME': 'en_US.UTF-8',
        'USER': 'tsai',
        'SSH_AGENT_PID': '6835',
        'XDG_SESSION_TYPE': 'x11',
        'SHLVL': '0',
        'HOME': '/home/tsai',
        'DESKTOP_SESSION': 'ubuntu',
        'GIO_LAUNCHED_DESKTOP_FILE': '/usr/share/applications/code.desktop',
        'GTK_MODULES': 'gail:atk-bridge',
        'GNOME_SHELL_SESSION_MODE': 'ubuntu',
        'LC_MONETARY': 'en_US.UTF-8',
        'MANAGERPID': '6519',
        'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus',
        'GIO_LAUNCHED_DESKTOP_FILE_PID': '13788',
        'IM_CONFIG_PHASE': '1',
        'LOGNAME': 'tsai',
        '_': '/usr/share/code/code',
        'JOURNAL_STREAM': '8:60909',
        'XDG_SESSION_CLASS': 'user',
        'USERNAME': 'tsai',
        'GNOME_DESKTOP_SESSION_ID': 'this-is-deprecated',
        'WINDOWPATH': '2',
        'PATH': '/home/tsai/.cargo/bin:/home/tsai/.local/bin:/home/

In [1]:
!export a='test'

In [10]:
import subprocess
a=subprocess.run(["sensors"])
a.stdout

TypeError: 'NoneType' object is not callable

In [15]:
import subprocess

p = subprocess.Popen('sensors', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print(line.decode())
    print("="*70)
retval = p.wait()

nvme-pci-0100

Adapter: PCI adapter

Composite:    +30.9°C  (low  =  -5.2°C, high = +79.8°C)

                       (crit = +84.8°C)



iwlwifi_1-virtual-0

Adapter: Virtual device

temp1:        +29.0°C  



k10temp-pci-00c3

Adapter: PCI adapter

Vcore:         1.48 V  

Vsoc:          1.09 V  

Tctl:         +37.5°C  

Tdie:         +37.5°C  

Tccd1:        +35.8°C  

Icore:        14.00 A  

Isoc:          8.25 A  





In [20]:
subprocess.Popen(['bash', '-c', 'sensors | grep -i icore'], stdout=subprocess.PIPE).communicate()[0].decode()

'Icore:        24.00 A  \n'